In [30]:
import pandas as pd
import numpy as np
from datetime import date
import os, sys
from pathlib import Path
from gs_quant.timeseries import settlement_price

from DataMaster import DataMaster

# ignore warnings SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

sys.path.insert(1, os.path.abspath(Path(os.getcwd()).parent))
sys.path.insert(1, os.path.abspath(Path(os.getcwd()).parent.parent))


In [2]:
# Sample DataFrame
data = [1.73,0,0,15.87,0,0, 24.34]
df = pd.DataFrame(data, columns=['values'])
df

,values
0,1.73
1,0.00
2,0.00
3,15.87
4,0.00
5,0.00
6,24.34


In [3]:
df.ewm(span=3, adjust=False).mean()

,values
0,1.730000
1,0.865000
2,0.432500
3,8.151250
4,4.075625
5,2.037812
6,13.188906


In [4]:
fromDate = date(2010,1,1)
toDate = date.today()
dm = DataMaster()
_df = dm.getData(
    tickers=['USDKRW'],
    datasetName='FXFORWARDPOINTS_IMM',
    datasetArgs={
        'start': fromDate,
    },
    multiCol=True
)

In [5]:
fd = pd.pivot_table(_df, values='expirationDate', index='date', columns='expiration')
fd.columns = [f"{col} fixingDate" for col in fd.columns]
sd = pd.pivot_table(_df, values='settlementDate', index='date', columns='expiration')
sd.columns = [f"{col} settlementDate" for col in sd.columns]
pt = pd.pivot_table(_df, values='forwardPoint', index='date', columns='expiration')
pt.columns = [f"{col} forwardPoint" for col in pt.columns]
df = _df.loc[_df['expiration'] == 'IMM1', ['spot', 'expiration', 'forwardPoint', 'expirationDate', 'settlementDate', 'spotSettlementDate']]
imm = pd.concat([df, fd, sd, pt], axis=1).reset_index()

In [14]:
imm['IMM1 fixingDate Shift'] = imm['IMM1 fixingDate'].shift(38)
imm['distanceIMM1'] = [(r['IMM1 fixingDate Shift'] - r['date']).days if r['IMM1 fixingDate Shift'] >= r['date'] else -(r['date'] - r['IMM1 fixingDate Shift']).days for i, r in imm.iterrows()]
imm['distanceIMM1'] = [(r['IMM1 fixingDate'] - r['date']).days if pd.isna(r['distanceIMM1']) else r['distanceIMM1'] for i, r in imm.iterrows()]
imm['IMM1-2'] = imm['IMM2 forwardPoint'] - imm['IMM1 forwardPoint']
imm['IMM2-3'] = imm['IMM3 forwardPoint'] - imm['IMM2 forwardPoint']

In [15]:
# imm['IMM Switch'] = imm['IMM1 settlementDate'].diff()
# imm['IMM Switch'] = [1 if not pd.isna(i.days) and i.days != 0 else 0 for i in imm['IMM Switch']]
imm

,date,spot,expiration,forwardPoint,expirationDate,settlementDate,spotSettlementDate,IMM1 fixingDate,IMM2 fixingDate,IMM3 fixingDate,...,IMM4 settlementDate,IMM1 forwardPoint,IMM2 forwardPoint,IMM3 forwardPoint,IMM4 forwardPoint,IMM1 fixingDate Shift,distanceIMM1,IMM1-2,IMM2-3,IMM Switch
0,2010-01-04,1149.000,IMM1,2.934,2010-03-15,2010-03-17,2010-01-06,2010-03-15,2010-06-14,2010-09-13,...,NaT,2.934,8.230,13.230,NaN,NaT,70.0,5.296,5.000,0
1,2010-01-05,1140.500,IMM1,2.980,2010-03-15,2010-03-17,2010-01-07,2010-03-15,2010-06-14,2010-09-13,...,NaT,2.980,8.197,13.480,NaN,NaT,69.0,5.217,5.283,0
2,2010-01-06,1130.000,IMM1,3.029,2010-03-15,2010-03-17,2010-01-08,2010-03-15,2010-06-14,2010-09-13,...,NaT,3.029,8.323,14.187,NaN,NaT,68.0,5.294,5.864,0
3,2010-01-07,1137.000,IMM1,2.700,2010-03-15,2010-03-17,2010-01-11,2010-03-15,2010-06-14,2010-09-13,...,NaT,2.700,8.043,13.598,NaN,NaT,67.0,5.343,5.555,0
4,2010-01-08,1124.000,IMM1,2.532,2010-03-15,2010-03-17,2010-01-12,2010-03-15,2010-06-14,2010-09-13,...,NaT,2.532,7.304,12.574,NaN,NaT,66.0,4.772,5.270,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3774,2024-09-04,1335.350,IMM1,-1.023,2024-09-12,2024-09-19,2024-09-06,2024-09-12,2024-12-16,2025-03-17,...,2025-06-18,-1.023,-7.900,-13.990,-18.529,2024-09-12,8.0,-6.877,-6.090,0
3775,2024-09-05,1332.345,IMM1,-0.820,2024-09-12,2024-09-19,2024-09-09,2024-09-12,2024-12-16,2025-03-17,...,2025-06-18,-0.820,-7.624,-13.760,-17.878,2024-09-12,7.0,-6.804,-6.136,0
3776,2024-09-06,1338.690,IMM1,-0.724,2024-09-12,2024-09-19,2024-09-10,2024-09-12,2024-12-16,2025-03-17,...,2025-06-18,-0.724,-7.517,-13.524,-17.686,2024-09-12,6.0,-6.793,-6.007,0
3777,2024-09-09,1341.700,IMM1,-0.637,2024-09-12,2024-09-19,2024-09-11,2024-09-12,2024-12-16,2025-03-17,...,2025-06-18,-0.637,-7.571,-13.265,-17.184,2024-09-12,3.0,-6.934,-5.694,0


In [23]:
# Get 1x3 points
ccy = 'TWD'
_df1 = dm.getData(
    tickers=[
        f"FX Forward {ccy.upper()}/USD 1m"
    ],
    datasetName='FXFORWARDPOINTS_V2_PREMIUM',
    datasetArgs={
        'start': fromDate,
        'pricingLocation': 'HKG'
    },
    multiCol=True
)
_df3 = dm.getData(
    tickers=[
        f"FX Forward {ccy.upper()}/USD 3m"
    ],
    datasetName='FXFORWARDPOINTS_V2_PREMIUM',
    datasetArgs={
        'start': fromDate,
        'pricingLocation': 'HKG'
    },
    multiCol=True
)

In [26]:
_df1 = _df.rename(columns={'forwardPoint': '1M forwardPoint'})
_df3 = _df2.rename(columns={'forwardPoint': '3M forwardPoint'})

,assetId,pricingLocation,csaTerms,fwdPoints,spot,fwdFxRate,settlementDate,updateTime,cross,settlementTenor
date,,,,,,,,,,
2010-01-04,MA5PDK5JST35H3Y8,HKG,USD-1,-0.1850,31.860,31.6750,2010-02-08,2022-07-19 02:04:35.450000+00:00,USDTWD,1m
2010-01-05,MA5PDK5JST35H3Y8,HKG,USD-1,-0.1750,31.820,31.6450,2010-02-08,2022-07-19 02:04:35.450000+00:00,USDTWD,1m
2010-01-06,MA5PDK5JST35H3Y8,HKG,USD-1,-0.1550,31.825,31.6700,2010-02-08,2022-07-19 02:04:35.450000+00:00,USDTWD,1m
2010-01-07,MA5PDK5JST35H3Y8,HKG,USD-1,-0.1350,31.850,31.7150,2010-02-11,2022-07-19 02:04:35.450000+00:00,USDTWD,1m
2010-01-08,MA5PDK5JST35H3Y8,HKG,USD-1,-0.1050,31.855,31.7500,2010-02-12,2022-07-19 02:04:35.450000+00:00,USDTWD,1m
...,...,...,...,...,...,...,...,...,...,...
2024-09-04,MA5PDK5JST35H3Y8,HKG,USD-1,-0.1000,32.145,32.0450,2024-10-07,2024-09-04 11:54:04.233000+00:00,USDTWD,1m
2024-09-05,MA5PDK5JST35H3Y8,HKG,USD-1,-0.1650,32.075,31.9100,2024-10-09,2024-09-05 12:12:03.995000+00:00,USDTWD,1m
2024-09-06,MA5PDK5JST35H3Y8,HKG,USD-1,-0.2000,31.893,31.6930,2024-10-11,2024-09-06 12:08:50.954000+00:00,USDTWD,1m


In [27]:
_df2

,assetId,pricingLocation,csaTerms,fwdPoints,spot,fwdFxRate,settlementDate,updateTime,cross,settlementTenor
date,,,,,,,,,,
2010-01-04,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.4450,31.860,31.4150,2010-04-06,2022-07-19 02:09:59.523000+00:00,USDTWD,3m
2010-01-05,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.4350,31.820,31.3850,2010-04-07,2022-07-19 02:09:59.523000+00:00,USDTWD,3m
2010-01-06,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.3950,31.825,31.4300,2010-04-08,2022-07-19 02:09:59.523000+00:00,USDTWD,3m
2010-01-07,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.3450,31.850,31.5050,2010-04-12,2022-07-19 02:09:59.523000+00:00,USDTWD,3m
2010-01-08,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.3300,31.855,31.5250,2010-04-12,2022-07-19 02:09:59.523000+00:00,USDTWD,3m
...,...,...,...,...,...,...,...,...,...,...
2024-09-04,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.3225,32.145,31.8225,2024-12-06,2024-09-04 11:54:04.233000+00:00,USDTWD,3m
2024-09-05,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.3975,32.075,31.6775,2024-12-09,2024-09-05 12:17:22.042000+00:00,USDTWD,3m
2024-09-06,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.4475,31.893,31.4455,2024-12-10,2024-09-06 12:14:59.880000+00:00,USDTWD,3m


In [28]:
_df3 = dm.getData(
    tickers=[
        f"FX Forward {ccy.upper()}/USD 1m", f"FX Forward {ccy.upper()}/USD 3m"
    ],
    datasetName='FXFORWARDPOINTS_V2_PREMIUM',
    datasetArgs={
        'start': fromDate,
        'pricingLocation': 'HKG'
    },
    multiCol=True
)

In [29]:
_df3

,assetId,pricingLocation,csaTerms,fwdPoints,spot,fwdFxRate,settlementDate,updateTime,cross,settlementTenor
date,,,,,,,,,,
2010-01-04,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.4450,31.860,31.4150,2010-04-06,2022-07-19 02:09:59.523000+00:00,USDTWD,3m
2010-01-05,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.4350,31.820,31.3850,2010-04-07,2022-07-19 02:09:59.523000+00:00,USDTWD,3m
2010-01-06,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.3950,31.825,31.4300,2010-04-08,2022-07-19 02:09:59.523000+00:00,USDTWD,3m
2010-01-07,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.3450,31.850,31.5050,2010-04-12,2022-07-19 02:09:59.523000+00:00,USDTWD,3m
2010-01-08,MAP9SMJJMGGZ40TV,HKG,USD-1,-0.3300,31.855,31.5250,2010-04-12,2022-07-19 02:09:59.523000+00:00,USDTWD,3m
...,...,...,...,...,...,...,...,...,...,...
2024-09-04,MA5PDK5JST35H3Y8,HKG,USD-1,-0.1000,32.145,32.0450,2024-10-07,2024-09-04 11:54:04.233000+00:00,USDTWD,1m
2024-09-05,MA5PDK5JST35H3Y8,HKG,USD-1,-0.1650,32.075,31.9100,2024-10-09,2024-09-05 12:12:03.995000+00:00,USDTWD,1m
2024-09-06,MA5PDK5JST35H3Y8,HKG,USD-1,-0.2000,31.893,31.6930,2024-10-11,2024-09-06 12:08:50.954000+00:00,USDTWD,1m
